## Internet Access in Louisville

In [100]:
import pandas as pd
import numpy as np

In [101]:
df = pd.read_csv("jeff_co_int.csv")

In [102]:
list(df)

['YEAR',
 'SAMPLE',
 'SERIAL',
 'CBSERIAL',
 'HHWT',
 'CLUSTER',
 'STATEFIP',
 'METRO',
 'MET2013',
 'PUMA',
 'STRATA',
 'GQ',
 'CINETHH',
 'CILAPTOP',
 'CISMRTPHN',
 'CITABLET',
 'CIHISPEED',
 'CIDIAL',
 'PERNUM',
 'PERWT',
 'SEX',
 'AGE',
 'RACE',
 'RACED',
 'HISPAN',
 'HISPAND',
 'EDUC',
 'EDUCD',
 'POVERTY',
 'city',
 'int_acc',
 'hspd_int',
 'computer',
 'tablet',
 'comp_tab',
 'hspd_dev']

There is no case_when in pandas. So it's either a nested else if, or just create a new column and fill in the values. I find filling in the values easier to read than a nested else if

In [103]:
# Start by creating a new column and filling it with NA values
df['hspd'] = np.nan

# Any time high speed internet is between 10 and 17 in the Census codebook it means they do have internet
df['hspd'].loc[((df['CIHISPEED'] >= 10) & (df['CIHISPEED'] <= 17))] = 1

# If they don't have internet at all and are NA for the high speed internet question then we want them marked as not having high speed internet, not as an NA value
df['hspd'].loc[((df['CINETHH'] == 3) & (df['CIHISPEED'] == 0))] = 0

# Finally if they say they don't have high speed internet we want them marked as not having high speed internet
df['hspd'].loc[df['CIHISPEED'] == 20] = 0


Now we need to use census weights to appropriately estimate the percentage of people with high speed internet in their households. 

In [104]:
# df18 = df.query('YEAR == 2018')
df18 = df[df.YEAR == 2018]

In [105]:
per_int = df18['PERWT'].loc[df18['hspd'] == 1].sum()
per_no_int = df18['PERWT'].loc[df18['hspd'] == 0].sum()
per_total = df18['PERWT'].sum()
per_int_percent = per_int/per_total

In [106]:
per_int_percent

0.7204147671636177

In [107]:
per_total

771035

In [108]:
year_totals = df.groupby(by = ['hspd', 'YEAR'], as_index = False)['PERWT'].sum() # if you don't use as_index = False you get a weird labeled series back that's hard to work with
year_totals

,hspd,YEAR,PERWT
0,0.0,2013,161299
1,0.0,2014,170809
2,0.0,2015,144626
3,0.0,2016,158520
4,0.0,2017,160849
5,0.0,2018,184795
6,1.0,2013,544770
7,1.0,2014,526480
8,1.0,2015,548358
9,1.0,2016,573787


Pivoting the dataframe is probably the easiest way to sum and get percentages now

In [114]:
year_totals.pivot(index = 'YEAR', columns = 'hspd', values = 'PERWT')

hspd,0.0,1.0
YEAR,,
2013,161299,544770
2014,170809,526480
2015,144626,548358
2016,158520,573787
2017,160849,577340
2018,184795,555465


In [117]:
year_totals['total'] = year_totals['hspd']['0.0'] + year_totals['hspd']['1.0']
year_totals

KeyError: '0.0'

In [110]:
year_totals['YEAR']

0     2013
1     2014
2     2015
3     2016
4     2017
5     2018
6     2013
7     2014
8     2015
9     2016
10    2017
11    2018
Name: YEAR, dtype: int64

In [111]:
year_totals.pivot(index = 'YEAR', columns = 'hspd', values = '')

KeyError: ''

In [112]:
df['hspd'].describe()


count    46480.000000
mean         0.808498
std          0.393487
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: hspd, dtype: float64

In [113]:
df['hspd_int'].describe()

count    46480.000000
mean         0.808498
std          0.393487
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: hspd_int, dtype: float64